In [7]:
import datetime
import importlib
from matplotlib import pyplot as plt
from data_processing import prom_wrapper, response_validator, metric_parser
from kad_utils import kad_utils
from model.sarima_model import SarimaModel
from visualization import visualization

In [8]:

PROMETHEUS_URL = "http://localhost:9090/"
METRIC_NAME = "rest_client_requests_total"
# TODO remove hardcodes
START_TIME = datetime.datetime.strptime("2021-03-23 21:10:00", "%Y-%m-%d %H:%M:%S")
END_TIME = datetime.datetime.strptime("2021-03-23 21:50:00", "%Y-%m-%d %H:%M:%S")
# LABEL_CONFIG = {"code": "200", "host": "kind-control-plane:6443", "instance": "kind-control-plane",
#                 "job": "kubernetes-nodes", "method": "PATCH"}
# TODO dictionary 2 prometheus labels format
LABEL_CONFIG_STR = "{code=\"200\",host=\"kind-control-plane:6443\",instance=\"kind-control-plane\"," \
                   "job=\"kubernetes-nodes\", method=\"PUT\"} "
QUERY = "rate(" + METRIC_NAME + LABEL_CONFIG_STR + "[1m])"

In [9]:
prom = prom_wrapper.PrometheusConnectWrapper(PROMETHEUS_URL)
steps_to_forecast = 10
sarima_model = SarimaModel(order=(0, 0, 0), seasonal_order=(1, 0, 1, 24))

metric_range = prom.perform_query(query=QUERY,
                                       start_time=START_TIME,
                                       end_time=END_TIME)

metric = response_validator.validate(metric_range)
metric_df = metric_parser.metric_to_dataframe(metric, METRIC_NAME).astype(float)
train_df, test_df = metric_parser.split_dataset(metric_df)

test_df = kad_utils.normalize(test_df, train_df.mean(), train_df.std())
train_df = kad_utils.normalize(train_df, train_df.mean(), train_df.std())

train_df[METRIC_NAME].plot()
test_df[METRIC_NAME].plot()
plt.show()

ConnectionError: HTTPConnectionPool(host='localhost', port=9090): Max retries exceeded with url: //api/v1/query_range?query=rate%28rest_client_requests_total%7Bcode%3D%22200%22%2Chost%3D%22kind-control-plane%3A6443%22%2Cinstance%3D%22kind-control-plane%22%2Cjob%3D%22kubernetes-nodes%22%2C+method%3D%22PUT%22%7D+%5B1m%5D%29&start=1616530200&end=1616532600&step=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa1321d86d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
sarima_model.train(train_df)

for i in range(steps_to_forecast, len(test_df), steps_to_forecast):
    result_df = sarima_model.test(test_df[i-steps_to_forecast:i])

In [ ]:
visualization = importlib.reload(visualization)
visualization.visualize(result_df, METRIC_NAME)